In [1]:

import os
import sys
import numpy as np
import neo
import pandas as pd
import h5py
import ast
import McsPy
import sys, importlib, os
import McsPy.McsData
import McsPy.McsCMOS
from McsPy import ureg, Q_
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz, find_peaks, correlate, gaussian, filtfilt
from scipy import stats
from scipy import signal
from scipy import stats
from scipy import signal
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import McsPy
import McsPy.McsData
from McsPy import ureg, Q_
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import scipy
import time
import glob
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import seaborn as sns
import copy
import pickle
import fnmatch

# Plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import seaborn as sns
#networkx
import plotly.graph_objects as go
import networkx as nx
import matplotlib.patches as mpatche

In [2]:
os.chdir('/Users/jonas/Documents/DATA/MEA_DATA_Aachen_sample/output__old/2021-05-17_cortex_div11_aCSF_ID046_30µMNE_spont_1')

In [3]:
pkl_list = glob.glob('*.pkl')

In [4]:
MAIN_RECORDING_DICTIONARY = pickle.load(
    open(pkl_list[0], 
         "rb"))



MAIN_RECORDING_DICTIONARY['Infos_Recording'].keys()
tick = MAIN_RECORDING_DICTIONARY['Infos_Recording']['tick']
timelengthrecording_s = MAIN_RECORDING_DICTIONARY['Infos_Recording']['timelengthrecording_s']
scale_factor_for_second = MAIN_RECORDING_DICTIONARY['Infos_Recording']['scale_factor_for_second']
scale_factor_for_milisecond = scale_factor_for_second/1000
spikedic_MAD = MAIN_RECORDING_DICTIONARY['spikedic_MAD']


network_bursts_seconds = MAIN_RECORDING_DICTIONARY['network_bursts_seconds']
spikedic_MAD = MAIN_RECORDING_DICTIONARY['spikedic_MAD']
fr_dic = MAIN_RECORDING_DICTIONARY['fr_dic']
isi_dictionary = MAIN_RECORDING_DICTIONARY['isi_dictionary']
isi_average_dic = MAIN_RECORDING_DICTIONARY['isi_average_dic']
Infos_Recording = MAIN_RECORDING_DICTIONARY['Infos_Recording']
Infos_Analysis = MAIN_RECORDING_DICTIONARY['Infos_Analysis']
Infos_Anatomy = MAIN_RECORDING_DICTIONARY['Infos_Anatomy']
Bursts = MAIN_RECORDING_DICTIONARY['Bursts']
Interburst_Intervals = MAIN_RECORDING_DICTIONARY['Interburst-Intervals']
bursting_time_per_channel = MAIN_RECORDING_DICTIONARY['bursting_time_per_channel']
bursts_per_channel = MAIN_RECORDING_DICTIONARY['bursts_per_channel']
burst_connections = MAIN_RECORDING_DICTIONARY['burst_connections']
Basics = MAIN_RECORDING_DICTIONARY['Basics']
inverted_layerdic = Infos_Anatomy['layerdic_invert']
layerdic = Infos_Anatomy['layerdic']
scale_factor_for_second = 1e-06
timelengthrecording_s = Infos_Recording['timelengthrecording_s']

infos_dic_subrecordings = Infos_Recording['info_dics_subrecordings']
throwkey = list(infos_dic_subrecordings.keys())[0]
subrec_infos = infos_dic_subrecordings[throwkey]

dict_keys(['filename', 'slice_id', 'medium', 'drug', 'stimulation', 'tissue', 'recording_date', 'info_dics_subrecordings', 'recordings_date', 'timelengthrecording_s', 'scale_factor_for_second', 'tick'])

## Create a function to sort the appearance of spikes in a network event

In [5]:
def sort_channels_by_leading_spike(spikedic, start_seconds, stop_seconds, scalefactor_for_second, tick):
    
        # first only find the relevant spikes for the searched time
        spikedic_only_relevant_spikes = {}
        for key in spikedic:
            spike_list = []
            for i in spikedic[key]:
                if start_seconds < (i*tick*scalefactor_for_second) < stop_seconds:
                    spike_list.append(i)
                spikedic_only_relevant_spikes[key] = spike_list
                
        #return spikedic_only_relevant_spikes
        # second: make a list of tuples with (first spike, channellabel)
        
        sortlist = []
        for key in spikedic_only_relevant_spikes:
            if len(spikedic_only_relevant_spikes[key]) > 0:
                   sortlist.append((spikedic_only_relevant_spikes[key][0], key))
                   
        #return sortlist
        # sort this list
        sortlist.sort(key=lambda x: x[0])
        
        return sortlist
        
        
        


In [6]:
d = sort_channels_by_leading_spike(spikedic = spikedic_MAD,
                                   start_seconds=network_bursts_seconds[15][0],
                                   stop_seconds=network_bursts_seconds[15][1],
                                   scalefactor_for_second=scale_factor_for_second,
                                   tick = tick)

In [7]:
d

[(1492655, 'K2'),
 (1492673, 'M2'),
 (1492689, 'L2'),
 (1492786, 'L1'),
 (1492832, 'K3'),
 (1492860, 'H3'),
 (1492911, 'J2'),
 (1492921, 'H1'),
 (1492945, 'H2'),
 (1492960, 'G1'),
 (1493034, 'G2'),
 (1493041, 'F2'),
 (1493121, 'F3'),
 (1494685, 'K4')]

## funktion: für alle vorhandenen netzwerkbursts ab start burst die zeit zum ersten spike pro kanal und diese dann nach mean sorten + spikeplot

In [8]:
def sort_channels_by_leading_spike_in_networkburst(spikedic, networkbursttimes_in_seconds, scale_factor_for_second, tick):
    
        # for each channel: 
        
        
        spikedic_times_start_after_networkburst_ms = {}
        for key in spikedic:
            start_list = []
            for i in networkbursttimes_in_seconds:
                start_ms = i[0]*1000
                stop_ms = i[1]*1000
                nb_spikelist = []
                #print(start_ms)
                
                for i in spikedic[key]:
                    #print(i)
                    s = i*tick*scale_factor_for_second*1000
                    if start_ms < s < stop_ms:
                        nb_spikelist.append(s)
                if len(nb_spikelist)>0:
                    first_spike_after_networkburst_start_ms = nb_spikelist[0]
                    diff_to_first_spike_ms = first_spike_after_networkburst_start_ms - start_ms
                    start_list.append(diff_to_first_spike_ms)
        
            spikedic_times_start_after_networkburst_ms[key] = start_list
            
        # return spikedic_times_start_after_networkburst_ms
        
        
        # second sort them after the mean difference in ms after start
        sortlist = []
        for key in spikedic_times_start_after_networkburst_ms:
            if len(spikedic_times_start_after_networkburst_ms[key]) > 0:
                   sortlist.append((np.mean(spikedic_times_start_after_networkburst_ms[key]), 
                                    np.std(spikedic_times_start_after_networkburst_ms[key]),
                                    key))

                    
                    
        # sort this list 
        
        sortlist.sort(key=lambda x: x[0])

        return sortlist, spikedic_times_start_after_networkburst_ms
        
        
        
        
        
        
        # sorted list of tuples mit (mean_time_from_networkburst_start, std, channellabel)
        

In [22]:
f, x = sort_channels_by_leading_spike_in_networkburst(spikedic = spikedic_MAD,
                                                   networkbursttimes_in_seconds=network_bursts_seconds[:],
                                                   scale_factor_for_second=scale_factor_for_second,
                                                   tick = tick)

In [23]:
def get_df_from_first_spike_dic(spikedic):
    
    
    inverted_dic = {}
    
    for key in spikedic:
        mean = np.mean(spikedic[key])
        for i in spikedic[key]:
            inverted_dic[i]=(key, mean)
            
    df = pd.DataFrame(data=list(inverted_dic.items()),columns=['spiketime', 'tuple'])
    df = df.assign(**dict(zip('cm', df.tuple.str)))
    #df = df.assign(channel = lambda df: df.tuple[0])
    #df = df.assign(mean = lambda df: df.tuple[1])
    df = df.sort_values(by='m')
            
    return inverted_dic , df

In [24]:
iv, i = get_df_from_first_spike_dic(x)

In [25]:
i

,spiketime,tuple,c,m
73,656.069483,"(E3, 682.9441530612205)",E3,682.944153
72,914.694850,"(E3, 682.9441530612205)",E3,682.944153
71,862.048944,"(E3, 682.9441530612205)",E3,682.944153
70,321.671213,"(E3, 682.9441530612205)",E3,682.944153
69,18.480000,"(E3, 682.9441530612205)",E3,682.944153
...,...,...,...,...
30,1866.549257,"(R6, 1217.0934252491606)",R6,1217.093425
50,1429.409993,"(B9, 1259.51673887043)",B9,1259.516739
49,1866.349257,"(B9, 1259.51673887043)",B9,1259.516739
19,1866.309257,"(F13, 1314.0361837209202)",F13,1314.036184


In [26]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import matplotlib.pyplot as plt
%matplotlib widget

In [27]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_theme(style="ticks")

# Initialize the figure with a logarithmic x axis
f, ax = plt.subplots(figsize=(16, 16))

# Load the example planets dataset

# Plot the orbital period with horizontal boxes
sns.boxplot(x='spiketime', y='c', data=i,
            width=.6, palette="vlag")

# Add in points to show each observation

sns.swarmplot(x='spiketime', y='c', data=i,
              size=4, color=".3", linewidth=0)

# Tweak the visual presentation
ax.xaxis.grid(True)
ax.set(ylabel="")
#sns.despine(trim=True, left=True)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='spiketime', ylabel='c'>

<AxesSubplot:xlabel='spiketime', ylabel='c'>

[Text(0, 0.5, '')]

In [15]:
data_with_ave = i.assign(
    average = i.groupby(1).mean())

KeyError: 1

In [185]:
i.groupby(1)


In [186]:
i

,0,1
256,2.800000,L2
196,7.840000,K2
156,8.480000,J2
215,11.720000,K3
239,11.840000,L1
...,...,...
9,1866.309257,F13
28,1866.349257,B9
20,1866.509257,D14
16,1866.549257,R6
